In [18]:
import numpy as np

# 活性化関数とその微分
## ReLU関数
定義
入力が0以下なら0、0より大きいならそのままの数値を返す
## ReLUの微分
定義
入力が正の領域では勾配1、0以下では勾配0となる
用途
逆伝播において、誤差を隠れ層へ伝播する際に使用する

In [20]:
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    # xが0より大きい部分は1、そうでなければ0を返す
    return np.where(x > 0, 1, 0)

# 損失関数
定義
予測値 y_predと正解値yとの差の二乗の平均を計算する
用途
回帰問題において、モデルの出力と実際の値との差を定量的に評価し、学習の指標とする

In [22]:
# 損失関数：平均二乗誤差 (MSE)
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# 2層ニューラルネットワークの実装
## 初期化(__init__)
重み(W1,W2)の初期化  
入力層 → 隠れ層、隠れ層 → 出力層への線形変換に必要な重みを乱数で初期化することで、学習開始時の対称性を防止し、効率的な学習を促す  
バイアス(b1,b2)の初期化  
重みと同様に、各層のバイアス項をゼロに初期化する  
## 順伝播(forward)
### 隠れ層の計算  
線形結合：z1 = X・W1+b1  
ReLU活性化：a1 = ReLU(z1)　非線形性を与えて、より複雑な関数を近似できるようにする  
### 出力層の計算  
線形結合のみ：z2 = a1・W2+b2　回帰問題の場合は、活性化関数を適用せずそのまま出力を返す  
## 逆伝播(backward)
### 出力層の誤差計算  
誤差：error_output = y_pred - y　損失関数の微分として、出力層での誤差を求める  
### 出力層のパラメータ勾配  
重みの勾配：dW2 = a1  
バイアスの勾配：db2 = np.sum(error_output, axis=0, keepdims=True)  
### 隠れ層の誤差
error_hidden = np.dot(error_output, self.W2.T) * relu_deriv(self.z1)  
出力層からの誤差を隠れ層に伝播し、ReLUの微分をかけることで各ニューロンの勾配を求める
### 隠れ層の勾配
重みの勾配：dW1 = np.dot(X.T, error_hidden)  
バイアスの勾配：db1 = np.sum(error_hidden, axis=0, keepdims=True)  
### パラメータ更新
各パラメータは、計算された勾配に学習率をかけた値を引く(勾配降下法)ことで更新する

In [24]:
class NN:
    def __init__(self, input_size, hidden_size, output_size):
        # 重みの初期化（乱数）
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))
    
    def forward(self, X):
        # 1層目の線形結合とReLUによる活性化
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = relu(self.z1)
        # 2層目（出力層）の線形結合（今回は活性化関数なし）
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        # 出力をそのまま返す（回帰の場合）
        return self.z2
    
    def backward(self, X, y, y_pred, learning_rate):
        # 出力層の誤差
        error_output = y_pred - y  # (バッチサイズ, output_size)
        # 隠れ層から出力層への勾配
        dW2 = np.dot(self.a1.T, error_output)
        db2 = np.sum(error_output, axis=0, keepdims=True)
        
        # 隠れ層の誤差（ReLUの微分を乗算）
        error_hidden = np.dot(error_output, self.W2.T) * relu_deriv(self.z1)
        # 入力層から隠れ層への勾配
        dW1 = np.dot(X.T, error_hidden)
        db1 = np.sum(error_hidden, axis=0, keepdims=True)
        
        # パラメータの更新
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1


# 学習ループ
## エポック数
1000回の学習ループを回して、ネットワークがデータに適合するようにパラメータを更新する
## 順伝播と損失計算
各エポックで入力データをネットワークに通し、予測値を得たあと、MSE損失を計算する
## 逆伝播
損失に基づいてパラメータの勾配を計算し、学習率に応じてパラメータを更新する　　

In [32]:
if __name__ == "__main__":
    # ダミーデータの作成（例：入力2次元、出力1次元）
    np.random.seed(0)
    X = np.random.randn(100, 2)
    # 簡単な線形関係にノイズを加えたもの
    y = np.dot(X, np.array([[2], [-3]])) + 1 + 0.1 * np.random.randn(100, 1)
    
    # ネットワークの初期化
    nn = NN(input_size=2, hidden_size=5, output_size=1)
    
    # 学習ループ
    epochs = 1000
    learning_rate = 0.001
    for epoch in range(epochs):
        # 順伝播
        y_pred = nn.forward(X)
        # 損失計算
        loss = mse_loss(y, y_pred)
        # 逆伝播
        nn.backward(X, y, y_pred, learning_rate)
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 10.1670
Epoch 100, Loss: 0.0126
Epoch 200, Loss: 0.0112
Epoch 300, Loss: 0.0105
Epoch 400, Loss: 0.0101
Epoch 500, Loss: 0.0098
Epoch 600, Loss: 0.0096
Epoch 700, Loss: 0.0094
Epoch 800, Loss: 0.0093
Epoch 900, Loss: 0.0092
